In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
from PIL import Image
import math
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add, Lambda
from keras.models import Model
from keras.optimizers import SGD, Adam
import keras.backend as K
import os
K.clear_session()
print("Running Tensorflow version", tf.__version__)
print("Keras is running on", K.backend(), "backend")

Running Tensorflow version 2.1.0
Keras is running on tensorflow backend


Using TensorFlow backend.


In [2]:
# problem with model with rounding 
'''
def roundingAlgo(x): 
    # first one that works with model_1 & model_2 
    # problem - this rounding function is slow: model_2 = 3 hours / epoch
    # comparison, model_0 = 20 mins / epoch
    # in addition, off by half with integer inputs (lower than actual value, e.g. floor(2) ≈ 1.5, floor(2.01) ≈ 2)
    # source: https://en.wikipedia.org/wiki/Floor_and_ceiling_functions#Continuity_and_series_expansions
    if True:
        result = x - 0.5
        for p in range(1, 7):
            result = result + K.sin(x * p * 2 * math.pi) / (p * math.pi)
    return result
# '''
'''     
def roundingAlgo(x):
    # second one that works with model_2 
    # problem - this rounding function is slower than first working algo: model_2 = 4,2 hours / epoch
    # comparison, model_0 = 20 mins / epoch
    # source: self
    return x - x % 1
# '''
# '''
def roundingAlgo(x): 
    # simplification of the first algo loop by simplifying the expression for range(1,7)
    # problem - rounding function is still slow = 2,5 hours / epoch
    # all non-speed problem of first algo still applies
    result = x - 0.5
    resultCos = K.cos(2 * math.pi * x)
    return result + K.sin(2 * math.pi * x) * (1 + resultCos) * (13 + 2 * resultCos - 18 * K.pow(resultCos, 2) - 32 * K.pow(resultCos, 3) + 80 * K.pow(resultCos, 4)) / 15
# '''
'''
def roundingAlgo(x): 
    # made to fool the engine to have a gradient
    return 0 * x + K.round(x)
# '''


# check https://github.com/keras-team/keras/issues/2218
# check https://github.com/keras-team/keras/issues/2221
# https://www.tensorflow.org/api_docs/python/tf/custom_gradient
class RoundClampQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundClampQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundClampQ7_12, self).build(input_shape)
    def call(self, X):
        return K.clip(roundingAlgo(X * 4096), -524288, 524287) / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundClampQ7_12, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class RoundClampQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundClampQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundClampQ3_4, self).build(input_shape)
    def call(self, X):
        return K.clip(roundingAlgo(X * 16), -128, 127) / 16.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundClampQ3_4, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
# TODO: change IdentityFinalLayer into the final layer specified in https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg
# Reference: https://github.com/BoXiao123/py-tiny-yolo-from-scratch/blob/master/main.py
# Reference: https://github.com/BoXiao123/py-tiny-yolo-from-scratch/blob/master/forward_region.py
# Reference: https://blog.paperspace.com/how-to-implement-a-yolo-v3-object-detector-from-scratch-in-pytorch-part-2/
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
def DBL(previousLayer, layerFilter, kernelSize=(3, 3), roundingFunction=Identity(), name=None):
    placeholder = ""
    if name is not None:
        placeholder = str(name)
    return roundingFunction(LeakyReLU(alpha=0.1)(roundingFunction(BatchNormalization(name="BatchNorm_"+placeholder)(Conv2D(filters=layerFilter, kernel_size=kernelSize, padding='same', name="Conv2D_"+placeholder)(previousLayer)))))
print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
# from pycocotools.coco import COCO
nms = 0
coco = 0
''' // comment on this line to enable/disable this block
# this block uses COCO annotation to find how many classification for learning (in case of updates)
dataDir='.'
dataType='train2017'
coco=COCO('{}/annotations/instances_{}.json'.format(dataDir,dataType))
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
# '''
if isinstance(nms, list):
    classificationClass = len(nms)
else:
    classificationClass = 80

print("Number of class classification is", classificationClass)
''' // comment on this line to enable/disable this block
# this block runs a sample of target data generation
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import os

imgIds = coco.getImgIds(catIds=coco.getCatIds(catNms=['person','dog','skateboard']));
img = coco.loadImgs(imgIds[int(len(imgIds) / 2)])[0]

%matplotlib inline
imgLoad = Image.open('{}/{}/{}'.format(dataDir, dataType, img['file_name']))
imgWRatio = 448 / imgLoad.size[0]
imgHRatio = 448 / imgLoad.size[1]
imgLoad = np.asarray(imgLoad.resize((448,448)))

print("Sample image input")

imgplot = plt.imshow(imgLoad)
plt.show()

annotations = coco.loadAnns(ids=coco.getAnnIds(imgIds=int(img['file_name'].split('.')[0])))
bbox_array = list(map(lambda x: x['bbox'], annotations))
cat_array = list(map(lambda x: x['category_id'], annotations))
bigBox_array = []
smolBox_array = []
toAppendArray = []
for x in range(len(bbox_array)):
    print("object", x)
    print(x, "-", bbox_array[x], "\tcategory", cat_array[x])
    toAppendArray = bbox_array[x]
    toAppendArray[0] = math.floor(toAppendArray[0] * imgWRatio * 100) / 100
    toAppendArray[1] = math.floor(toAppendArray[1] * imgHRatio * 100) / 100
    toAppendArray[2] = math.floor(toAppendArray[2] * imgWRatio * 100) / 100
    toAppendArray[3] = math.floor(toAppendArray[3] * imgHRatio * 100) / 100
    if bbox_array[x][2] * bbox_array[x][3] > 10000:
        if len(bigBox_array) < 3:
            bigBox_array.append([cat_array[x], toAppendArray])
    else:
        if len(smolBox_array) < 3:
            smolBox_array.append([cat_array[x], toAppendArray])
# https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
# https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch
# https://medium.com/oracledevs/final-layers-and-loss-functions-of-single-stage-detectors-part-1-4abbfa9aa71c
print('Big Box')
for p in range(len(bigBox_array)):
    print(bigBox_array[p])
print('Small Box')
for p in range(len(smolBox_array)):
    print(smolBox_array[p])


# bigBox get small grid
smallGrid = np.zeros(shape=(14, 14, 3 * (5 + classificationClass)), dtype=np.float32)
for x in bigBox_array:
    for p in range(3):
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass)] = 1
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 1] = x[1][0]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 2] = x[1][1]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 3] = x[1][2]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 4] = x[1][3]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 4 + x[0]] = 1
# smolBox get big grid
bigGrid = np.zeros(shape=(28, 28, 3 * (5 + classificationClass)), dtype=np.float32)
for x in smolBox_array:
    for p in range(3):
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass)] = 1
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 1] = x[1][0]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 2] = x[1][1]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 3] = x[1][2]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 4] = x[1][3]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 4 + x[0]] = 1
        
print(K.constant(value=smallGrid, dtype='float32', shape=(14, 14, 3 * (classificationClass + 5))).shape)
print(K.constant(value=bigGrid, dtype='float32', shape=(28, 28, 3 * (classificationClass + 5))).shape)
i = 0
with (os.scandir('{}/{}/'.format(dataDir, dataType))) as trainingPictures:
    for currentPicture in trainingPictures:
        print(currentPicture.name)
        i = i + 1
        if i > 9:
            break
# '''

del coco

Number of class classification is 80


In [4]:
print()
''' // comment on this line to enable/disable this block
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import os
# '''

def rand(a=0, b=1):
    return np.random.rand()*(b-a) + a

def get_random_data(annotation_line, input_shape, random=True, max_boxes=20, jitter=.3, hue=.1, sat=1.5, val=1.5, proc_img=True):
    '''
    random preprocessing for real-time data augmentation 
    
    random=True induces image processing (better data accuracy with cost of cycles)
    '''
    line = annotation_line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = input_shape
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

    if not random:
        # resize image
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image_data=0
        if proc_img:
            image = image.resize((nw,nh), Image.BICUBIC)
            new_image = Image.new('RGB', (w,h), (128,128,128))
            new_image.paste(image, (dx, dy))
            image_data = np.array(new_image)/255.

        # correct boxes
        box_data = np.zeros((max_boxes,5))
        if len(box)>0:
            np.random.shuffle(box)
            if len(box)>max_boxes: box = box[:max_boxes]
            box[:, [0,2]] = box[:, [0,2]]*scale + dx
            box[:, [1,3]] = box[:, [1,3]]*scale + dy
            box_data[:len(box)] = box

        return image_data, box_data

    # resize image
    new_ar = w/h * rand(1-jitter,1+jitter)/rand(1-jitter,1+jitter)
    scale = rand(.25, 2)
    if new_ar < 1:
        nh = int(scale*h)
        nw = int(nh*new_ar)
    else:
        nw = int(scale*w)
        nh = int(nw/new_ar)
    image = image.resize((nw,nh), Image.BICUBIC)

    # place image
    dx = int(rand(0, w-nw))
    dy = int(rand(0, h-nh))
    new_image = Image.new('RGB', (w,h), (128,128,128))
    new_image.paste(image, (dx, dy))
    image = new_image

    # flip image or not
    flip = rand()<.5
    if flip: image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # distort image
    hue = rand(-hue, hue)
    sat = rand(1, sat) if rand()<.5 else 1/rand(1, sat)
    val = rand(1, val) if rand()<.5 else 1/rand(1, val)
    x = rgb_to_hsv(np.array(image)/255.)
    x[..., 0] += hue
    x[..., 0][x[..., 0]>1] -= 1
    x[..., 0][x[..., 0]<0] += 1
    x[..., 1] *= sat
    x[..., 2] *= val
    x[x>1] = 1
    x[x<0] = 0
    image_data = hsv_to_rgb(x) # numpy array, 0 to 1

    # correct boxes
    box_data = np.zeros((max_boxes,5))
    if len(box)>0:
        np.random.shuffle(box)
        box[:, [0,2]] = box[:, [0,2]]*nw/iw + dx
        box[:, [1,3]] = box[:, [1,3]]*nh/ih + dy
        if flip: box[:, [0,2]] = w - box[:, [2,0]]
        box[:, 0:2][box[:, 0:2]<0] = 0
        box[:, 2][box[:, 2]>w] = w
        box[:, 3][box[:, 3]>h] = h
        box_w = box[:, 2] - box[:, 0]
        box_h = box[:, 3] - box[:, 1]
        box = box[np.logical_and(box_w>1, box_h>1)] # discard invalid box
        if len(box)>max_boxes: box = box[:max_boxes]
        box_data[:len(box)] = box

    return image_data, box_data

def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format

    Parameters
    ----------
    true_boxes: array, shape=(m, T, 5)
        Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
    input_shape: array-like, hw, multiples of 32
    anchors: array, shape=(N, 2), wh
    num_classes: integer

    Returns
    -------
    y_true: list of array, shape like yolo_outputs, xywh are reletive value

    '''
    assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
    num_layers = len(anchors)//3 # default setting
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]

    true_boxes = np.array(true_boxes, dtype='float32')
    input_shape = np.array(input_shape, dtype='int32')
    boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
    boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
    true_boxes[..., 0:2] = boxes_xy/input_shape[::-1]
    true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

    m = true_boxes.shape[0]
    grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]

    # Expand dim to apply broadcasting.
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0]>0

    for b in range(m):
        # Discard zero rows.
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh)==0: continue
        # Expand dim to apply broadcasting.
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # Find best anchor for each true box
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b,t, 4].astype('int32')
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5+c] = 1

    return y_true

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i % n], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i + 1) % (n * 2)
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)
'''
# added quick-fix for categories that are not included in COCO dataset (2017) - https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
def cocoCategoryQF(value):
    subtractValue = 0
    if value > 12:
        subtractValue = subtractValue + 1
        if value > 26:
            subtractValue = subtractValue + 1
            if value > 30:
                subtractValue = subtractValue + 2
                if value > 45:
                    subtractValue = subtractValue + 1
                    if value > 66:
                        subtractValue = subtractValue + 1
                        if value > 69:
                            subtractValue = subtractValue + 2
                            if value > 71:
                                subtractValue = subtractValue + 1
                                if value > 83:
                                    subtractValue = subtractValue + 1
                                    if value > 91:
                                        subtractValue = subtractValue + 1
    return value - subtractValue
# '''
print("Target data generator successfully defined")


Target data generator successfully defined


In [5]:
# diambil dari https://github.com/qqwweee/keras-yolo3
'''
MIT License

Copyright (c) 2018 qqwweee

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

image_size = (416,416)
image_height, image_width = image_size

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

tinyYolo_anchors = get_anchors("../CNN-VLSI/tiny_yolo_anchors.txt")

def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs

def box_iou(b1, b2):
    '''Return iou tensor

    Parameters
    ----------
    b1: tensor, shape=(i1,...,iN, 4), xywh
    b2: tensor, shape=(j, 4), xywh

    Returns
    -------
    iou: tensor, shape=(i1,...,iN, j)

    '''

    # Expand dim to apply broadcasting.
    b1 = K.expand_dims(b1, -2)
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    b1_wh_half = b1_wh/2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half

    # Expand dim to apply broadcasting.
    b2 = K.expand_dims(b2, 0)
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh/2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half

    intersect_mins = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return iou

def yolo_loss(args, anchors, num_classes, ignore_thresh=.5, print_loss=False):
    '''Return yolo_loss tensor

    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss

    Returns
    -------
    loss: tensor, shape=(1,)

    '''
    num_layers = len(anchors)//3 # default setting
    yolo_outputs = args[:num_layers]
    y_true = args[num_layers:]
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]
    input_shape = K.cast(K.shape(yolo_outputs[0])[1:3] * 32, K.dtype(y_true[0]))
    grid_shapes = [K.cast(K.shape(yolo_outputs[l])[1:3], K.dtype(y_true[0])) for l in range(num_layers)]
    loss = 0
    m = K.shape(yolo_outputs[0])[0] # batch size, tensor
    mf = K.cast(m, K.dtype(yolo_outputs[0]))

    for l in range(num_layers):
        object_mask = y_true[l][..., 4:5]
        true_class_probs = y_true[l][..., 5:]

        grid, raw_pred, pred_xy, pred_wh = yolo_head(yolo_outputs[l], anchors[anchor_mask[l]], num_classes, input_shape, calc_loss=True)
        pred_box = K.concatenate([pred_xy, pred_wh])

        # Darknet raw box to calculate loss.
        raw_true_xy = y_true[l][..., :2]*grid_shapes[l][::-1] - grid
        raw_true_wh = K.log(y_true[l][..., 2:4] / anchors[anchor_mask[l]] * input_shape[::-1])
        raw_true_wh = K.switch(object_mask, raw_true_wh, K.zeros_like(raw_true_wh)) # avoid log(0)=-inf
        box_loss_scale = 2 - y_true[l][...,2:3]*y_true[l][...,3:4]

        # Find ignore mask, iterate over each of batch.
        ignore_mask = tf.TensorArray(K.dtype(y_true[0]), size=1, dynamic_size=True)
        object_mask_bool = K.cast(object_mask, 'bool')
        def loop_body(b, ignore_mask):
            true_box = tf.boolean_mask(y_true[l][b,...,0:4], object_mask_bool[b,...,0])
            iou = box_iou(pred_box[b], true_box)
            best_iou = K.max(iou, axis=-1)
            ignore_mask = ignore_mask.write(b, K.cast(best_iou<ignore_thresh, K.dtype(true_box)))
            return b+1, ignore_mask
        _, ignore_mask = tf.while_loop(lambda b,*args: b<m, loop_body, [0, ignore_mask])
        ignore_mask = ignore_mask.stack()
        ignore_mask = K.expand_dims(ignore_mask, -1)

        # K.binary_crossentropy is helpful to avoid exp overflow.
        xy_loss = object_mask * box_loss_scale * K.binary_crossentropy(raw_true_xy, raw_pred[...,0:2], from_logits=True)
        wh_loss = object_mask * box_loss_scale * 0.5 * K.square(raw_true_wh-raw_pred[...,2:4])
        confidence_loss = object_mask * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True)+ \
            (1-object_mask) * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True) * ignore_mask
        class_loss = object_mask * K.binary_crossentropy(true_class_probs, raw_pred[...,5:], from_logits=True)

        xy_loss = K.sum(xy_loss) / mf
        wh_loss = K.sum(wh_loss) / mf
        confidence_loss = K.sum(confidence_loss) / mf
        class_loss = K.sum(class_loss) / mf
        loss += xy_loss + wh_loss + confidence_loss + class_loss
        if print_loss:
            loss = tf.Print(loss, [loss, xy_loss, wh_loss, confidence_loss, class_loss, K.sum(ignore_mask)], message='loss: ')
    return loss

In [6]:
with open("../CNN-VLSI/train.txt") as trainText:
    train_annotation_lines = trainText.readlines()
with open("../CNN-VLSI/val.txt") as valText:
    val_annotation_lines = valText.readlines()
lenTrain = len(train_annotation_lines)
print("# of training data is", lenTrain)
lenVal = len(val_annotation_lines)
print("# of validation data is", lenVal)
np.random.seed(31415)
np.random.shuffle(train_annotation_lines)
np.random.shuffle(val_annotation_lines)

# of training data is 117266
# of validation data is 4952


In [7]:
model_0 = None
# '''
# model_0 does no rounding (float32 operation)
model_0_input = Input(shape=(None, None, 3), name="model_0_inputLayer")
model_0_pointer = model_0_input
print("Input shape:", model_0_pointer.shape) # 448 x 448 x 3
model_0_startBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_input, layerFilter=16, name="model_0_layer0_branch") 
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 448 x 448 x 16
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 224 x 224 x 16
model_0_startBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_startBranch, layerFilter=32, name="model_0_layer1_branch")
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 224 x 224 x 32
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 112 x 112 x 32
model_0_startBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_startBranch, layerFilter=64, name="model_0_layer2_branch")
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 112 x 112 x 64
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 56 x 56 x 64
model_0_startBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_startBranch, layerFilter=128, name="model_0_layer3_branch")
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 56 x 56 x 128
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 28 x 28 x 128
model_0_startBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_startBranch, layerFilter=256, name="model_0_layer4_branch")
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_0_branch0
model_0_branch0 = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 256
model_0_branch0 = DBL(roundingFunction=Identity(), previousLayer=model_0_branch0, layerFilter=512, name="model_0_layer5_branch0")
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_0_branch0)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = DBL(roundingFunction=Identity(), previousLayer=model_0_branch0, layerFilter=1024, name="model_0_layer6_branch0")
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 1024
model_0_branch0 = DBL(roundingFunction=Identity(), previousLayer=model_0_branch0, layerFilter=256, kernelSize=(1, 1))
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_0_branch0 (14 x 14 x 256), following model_0_branch00
model_0_branch00 = DBL(roundingFunction=Identity(), previousLayer=model_0_branch0, layerFilter=128, kernelSize=(1, 1), name="model_0_layer7_branch00")
model_0_pointer = model_0_branch00
print(model_0_pointer.shape) # 14 x 14 x 128
model_0_branch00 = UpSampling2D()(model_0_branch00)
model_0_pointer = model_0_branch00
print(model_0_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_0_branch1 (unchanged from model_0_startBranch) and model_0_branch00
model_0_mergedBranch = Concatenate()([model_0_startBranch, model_0_branch00])
model_0_pointer = model_0_mergedBranch
print(model_0_pointer.shape) # 28 x 28 x 384
model_0_mergedBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_mergedBranch, layerFilter=256, name="model_0_layer8_branch1")
model_0_pointer = model_0_mergedBranch
print(model_0_pointer.shape) # 28 x 28 x 256
model_0_mergedBranch = DBL(roundingFunction=Identity(), previousLayer=model_0_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass), name="model_0_layer9_branch1")
model_0_pointer = model_0_mergedBranch
print("Model output 0 shape:", model_0_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_0_mergedBranch = IdentityFinalLayer(name="model_0_outputLayer_0")(model_0_mergedBranch)
print() # OUTPUT = model_0_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_0_branch01
model_0_branch01 = DBL(roundingFunction=Identity(), previousLayer=model_0_branch0, layerFilter=512, name="model_0_layer7_branch01")
model_0_pointer = model_0_branch01
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch01 = DBL(roundingFunction=Identity(), previousLayer=model_0_branch01, layerFilter=3 * (4 + 1 + classificationClass), name="model_0_layer8_branch01")
model_0_pointer = model_0_branch01
print("Model output 1 shape:", model_0_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_0_branch01 = IdentityFinalLayer(name="model_0_outputLayer_1")(model_0_branch01)
print() # OUTPUT = model_0_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_0_actual = Model(inputs=model_0_input, outputs=[model_0_mergedBranch, model_0_branch01])
# model_0.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])

# added code from https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py
setArgs_model_0 = {
    'anchors': tinyYolo_anchors, 
    'num_classes': classificationClass, 
    'ignore_thresh': 0.7
}

y_true_model_0 = [Input(shape=(image_height//{0:32, 1:16}[l], image_width//{0:32, 1:16}[l], 3, classificationClass + 5)) for l in range(2)]

model_0_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss_model_0', arguments=setArgs_model_0)([*model_0_actual.output, *y_true_model_0])

model_0 = Model([model_0_actual.input, *y_true_model_0], model_0_loss)

# model_0_actual adalah working model yang akan dipakai untuk inference
# y_true adalah layer input yang akan diisikan nilai sebenarnya
# cara training model_0 adalah pendekatan sehingga output model_0 sedekat mungkin dengan 0 (model_0 ≈ model_0_actual - y_true)

model_0.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss_model_0': lambda y_true, y_pred: y_pred})

# loss adalah seberapa jauh nilai perbedaan output dengan 0

print("Model model_0 compilation complete") # tinggal porting line 54 sampai 61 dari https://github.com/awe777/keras-yolo3/blob/master/train.py, dilakukan di cell training di bawah
# '''
print()

Input shape: (None, None, None, 3)
(None, None, None, 16)
(None, None, None, 16)
(None, None, None, 32)
(None, None, None, 32)
(None, None, None, 64)
(None, None, None, 64)
(None, None, None, 128)
(None, None, None, 128)
(None, None, None, 256)
Branch split from main branch - following branch 0
(None, None, None, 256)
(None, None, None, 512)
(None, None, None, 512)
(None, None, None, 1024)
(None, None, None, 256)
Branch split from branch 0 - following branch 0,0
(None, None, None, 128)
(None, None, None, 128)
Branch merge from branch 1 and branch 0,0
(None, None, None, 384)
(None, None, None, 256)
Model output 0 shape: (None, None, None, 255)

Branch split from branch 0 - following branch 0,1
(None, None, None, 512)
Model output 1 shape: (None, None, None, 255)

Model model_0 compilation complete



In [8]:
# model_1 approximates Q7.12 signed fixed point operations with floating point rules (overflow = maximum/minimum value) 
# Done by rounding to the nearest 1/4096 and capping at [-128, 128) after batch normalization and activation layers
model_1 = None
'''
model_1_input = Input(shape=(448, 448, 3))
model_1_pointer = model_1_input
print("Input shape:", model_1_pointer.shape) # 448 x 448 x 3
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_input, layerFilter=16) 
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 448 x 448 x 16
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 224 x 224 x 16
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=32)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 224 x 224 x 32
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 112 x 112 x 32
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=64)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 112 x 112 x 64
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 56 x 56 x 64
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=128)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 56 x 56 x 128
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 28 x 28 x 128
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=256)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_1_branch0
model_1_branch0 = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 256
model_1_branch0 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=512)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_1_branch0)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=1024)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 1024
model_1_branch0 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=256, kernelSize=(1, 1))
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_1_branch0 (14 x 14 x 256), following model_1_branch00
model_1_branch00 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=128, kernelSize=(1, 1))
model_1_pointer = model_1_branch00
print(model_1_pointer.shape) # 14 x 14 x 128
model_1_branch00 = UpSampling2D()(model_1_branch00)
model_1_pointer = model_1_branch00
print(model_1_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_1_branch1 (unchanged from model_1_startBranch) and model_1_branch00
model_1_mergedBranch = Concatenate()([model_1_startBranch, model_1_branch00])
model_1_pointer = model_1_mergedBranch
print(model_1_pointer.shape) # 28 x 28 x 384
model_1_mergedBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_mergedBranch, layerFilter=256)
model_1_pointer = model_1_mergedBranch
print(model_1_pointer.shape) # 28 x 28 x 256
model_1_mergedBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_1_pointer = model_1_mergedBranch
print("Model output 0 shape:", model_1_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_1_mergedBranch = IdentityFinalLayer()(model_1_mergedBranch)
print() # OUTPUT = model_1_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1") # following model_1_branch01
model_1_branch01 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=512)
model_1_pointer = model_1_branch01
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch01 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_1_pointer = model_1_branch01
print("Model output 1 shape:", model_1_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_1_branch01 = IdentityFinalLayer()(model_1_branch01)
print() # OUTPUT = model_1_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_1 = Model(inputs=model_1_input, outputs=[model_1_mergedBranch, model_1_branch01])
model_1.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_1 compilation complete")
# '''
print()

In [9]:
# model_2 approximates Q7.12 signed fixed point operations with integer rules (overflow = positive -> negative & vice versa) 
# Done by rounding to the nearest 1/4096 and capping at [-128, 128) after batch normalization and activation layers
model_2 = None
'''
model_2_input = Input(shape=(448, 448, 3))
model_2_pointer = model_2_input
print("Input shape:", model_2_pointer.shape) # 448 x 448 x 3
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_input, layerFilter=16) 
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 448 x 448 x 16
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 224 x 224 x 16
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=32)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 224 x 224 x 32
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 112 x 112 x 32
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=64)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 112 x 112 x 64
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 56 x 56 x 64
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=128)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 56 x 56 x 128
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 28 x 28 x 128
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=256)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_2_branch0
model_2_branch0 = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 256
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=512)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_2_branch0)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=1024)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 1024
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=256, kernelSize=(1, 1))
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_2_branch0 (14 x 14 x 256), following model_2_branch00
model_2_branch00 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=128, kernelSize=(1, 1))
model_2_pointer = model_2_branch00
print(model_2_pointer.shape) # 14 x 14 x 128
model_2_branch00 = UpSampling2D()(model_2_branch00)
model_2_pointer = model_2_branch00
print(model_2_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_2_branch1 (unchanged from model_2_startBranch) and model_2_branch00
model_2_mergedBranch = Concatenate()([model_2_startBranch, model_2_branch00])
model_2_pointer = model_2_mergedBranch
print(model_2_pointer.shape) # 28 x 28 x 384
model_2_mergedBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_mergedBranch, layerFilter=256)
model_2_pointer = model_2_mergedBranch
print(model_2_pointer.shape) # 28 x 28 x 256
model_2_mergedBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_2_pointer = model_2_mergedBranch
print("Model output 0 shape:", model_2_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_2_mergedBranch = IdentityFinalLayer()(model_2_mergedBranch)
print() # OUTPUT = model_2_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_2_branch01
model_2_branch01 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=512)
model_2_pointer = model_2_branch01
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch01 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_2_pointer = model_2_branch01
print("Model output 1 shape:", model_2_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_2_branch01 = IdentityFinalLayer()(model_2_branch01)
print() # OUTPUT = model_2_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_2 = Model(inputs=model_2_input, outputs=[model_2_mergedBranch, model_2_branch01])
model_2.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_2 compilation complete")
# '''
print()

In [10]:
# model_3 approximates Q3.4 signed fixed point operations with floating point rules (overflow = maximum/minimum value) 
# Done by rounding to the nearest 1/16 and capping at [-8, 8) after batch normalization and activation layers
model_3 = None
'''
model_3_input = Input(shape=(448, 448, 3))
model_3_pointer = model_3_input
print("Input shape:", model_3_pointer.shape) # 448 x 448 x 3
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_input, layerFilter=16) 
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 448 x 448 x 16
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 224 x 224 x 16
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=32)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 224 x 224 x 32
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 112 x 112 x 32
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=64)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 112 x 112 x 64
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 56 x 56 x 64
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=128)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 56 x 56 x 128
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 28 x 28 x 128
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=256)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_3_branch0
model_3_branch0 = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 256
model_3_branch0 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=512)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_3_branch0)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch0 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=1024)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 1024
model_3_branch0 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=256, kernelSize=(1, 1))
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_3_branch0 (14 x 14 x 256), following model_3_branch00
model_3_branch00 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=128, kernelSize=(1, 1))
model_3_pointer = model_3_branch00
print(model_3_pointer.shape) # 14 x 14 x 128
model_3_branch00 = UpSampling2D()(model_3_branch00)
model_3_pointer = model_3_branch00
print(model_3_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_3_branch1 (unchanged from model_3_startBranch) and model_3_branch00
model_3_mergedBranch = Concatenate()([model_3_startBranch, model_3_branch00])
model_3_pointer = model_3_mergedBranch
print(model_3_pointer.shape) # 28 x 28 x 384
model_3_mergedBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_mergedBranch, layerFilter=256)
model_3_pointer = model_3_mergedBranch
print(model_3_pointer.shape) # 28 x 28 x 256
model_3_mergedBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_3_pointer = model_3_mergedBranch
print("Model output 0 shape:", model_3_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_3_mergedBranch = IdentityFinalLayer()(model_3_mergedBranch)
print() # OUTPUT = model_3_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1") # following model_3_branch01
model_3_branch01 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=512)
model_3_pointer = model_3_branch01
print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch01 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_3_pointer = model_3_branch01
print("Model output 1 shape:", model_3_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_3_branch01 = IdentityFinalLayer()(model_3_branch01)
print() # OUTPUT = model_3_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_3 = Model(inputs=model_3_input, outputs=[model_3_mergedBranch, model_3_branch01])
model_3.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_3 compilation complete")
# '''
print()

In [11]:
# model_4 approximates Q3.4 signed fixed point operations with integer rules (overflow = positive -> negative & vice versa) 
# Done by rounding to the nearest 1/16 and capping at [-8, 8) after batch normalization and activation layers
model_4 = None
'''
model_4_input = Input(shape=(448, 448, 3))
model_4_pointer = model_4_input
print("Input shape:", model_4_pointer.shape) # 448 x 448 x 3
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_input, layerFilter=16) 
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 448 x 448 x 16
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 224 x 224 x 16
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=32)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 224 x 224 x 32
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 112 x 112 x 32
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=64)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 112 x 112 x 64
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 56 x 56 x 64
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=128)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 56 x 56 x 128
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 28 x 28 x 128
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=256)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_4_branch0
model_4_branch0 = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 256
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=512)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_4_branch0)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=1024)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 1024
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=256, kernelSize=(1, 1))
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_4_branch0 (14 x 14 x 256), following model_4_branch00
model_4_branch00 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=128, kernelSize=(1, 1))
model_4_pointer = model_4_branch00
print(model_4_pointer.shape) # 14 x 14 x 128
model_4_branch00 = UpSampling2D()(model_4_branch00)
model_4_pointer = model_4_branch00
print(model_4_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_4_branch1 (unchanged from model_4_startBranch) and model_4_branch00
model_4_mergedBranch = Concatenate()([model_4_startBranch, model_4_branch00])
model_4_pointer = model_4_mergedBranch
print(model_4_pointer.shape) # 28 x 28 x 384
model_4_mergedBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_mergedBranch, layerFilter=256)
model_4_pointer = model_4_mergedBranch
print(model_4_pointer.shape) # 28 x 28 x 256
model_4_mergedBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_4_pointer = model_4_mergedBranch
print("Model output 0 shape:", model_4_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_4_mergedBranch = IdentityFinalLayer()(model_4_mergedBranch)
print() # OUTPUT = model_4_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_4_branch01
model_4_branch01 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=512)
model_4_pointer = model_4_branch01
print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch01 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_4_pointer = model_4_branch01
print("Model output 1 shape:", model_4_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_4_branch01 = IdentityFinalLayer()(model_4_branch01)
print() # OUTPUT = model_4_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_4 = Model(inputs=model_4_input, outputs=[model_4_mergedBranch, model_4_branch01])
model_4.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_4 compilation complete")
# '''
print()

In [12]:
if model_0 is not None:
    print(model_0.summary()) 

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
model_0_inputLayer (InputLayer) (None, None, None, 3 0                                            
__________________________________________________________________________________________________
Conv2D_model_0_layer0_branch (C (None, None, None, 1 448         model_0_inputLayer[0][0]         
__________________________________________________________________________________________________
BatchNorm_model_0_layer0_branch (None, None, None, 1 64          Conv2D_model_0_layer0_branch[0][0
__________________________________________________________________________________________________
identity_2 (Identity)           (None, None, None, 1 0           BatchNorm_model_0_layer0_branch[0
                                                                 leaky_re_lu_1[0][0]        

In [13]:
if model_1 is not None:
    print(model_1.summary())

In [14]:
if model_2 is not None:
    print(model_2.summary())

In [15]:
if model_3 is not None:
    print(model_3.summary())

In [16]:
if model_4 is not None:
    print(model_4.summary())

In [17]:
print()
trainingBatchSize = 16
train_data_generator = data_generator_wrapper(
    annotation_lines=train_annotation_lines, 
    batch_size=trainingBatchSize, 
    input_shape=image_size, 
    anchors=tinyYolo_anchors, 
    num_classes=classificationClass
)
val_data_generator = data_generator_wrapper(
    annotation_lines=val_annotation_lines, 
    batch_size=trainingBatchSize, 
    input_shape=image_size, 
    anchors=tinyYolo_anchors, 
    num_classes=classificationClass
)
print()

In [18]:
print()
# ''' // comment on this line to enable/disable this block
model_0_history = model_0.fit_generator(
    generator=train_data_generator, 
    steps_per_epoch=max(1, lenTrain//trainingBatchSize), 
    epochs=5,
    validation_data=val_data_generator,
    validation_steps=max(1, lenVal//trainingBatchSize),
)
# '''
print()


Epoch 1/5


InvalidArgumentError:  Incompatible shapes: [16,26,26,3,1] vs. [16,13,13,3,1]
	 [[node yolo_loss_model_0/logistic_loss_5/mul (defined at b:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_13960]

Function call stack:
keras_scratch_graph


In [ ]:
print()
''' // comment on this line to enable/disable this block
model_1_history = model_1.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()

In [ ]:
print()
''' // comment on this line to enable/disable this block
model_2_history = model_2.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()

In [ ]:
print()
''' // comment on this line to enable/disable this block
model_3_history = model_3.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()

In [ ]:
print()
''' // comment on this line to enable/disable this block
model_4_history = model_4.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()

In [ ]:
print("Resource successfully released")